# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Used to navigate to project root for api_keys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Load saved city weather data for analysis
city_weather_data_csv = '../WeatherPy/output_data/cities.csv'

# Read and display the CSV with Pandas
city_weather_data = pd.read_csv(city_weather_data_csv)
city_weather_data.head()

,city_name,city_id,lat,lon,unix_dt,temp_max_c,temp_max_f,humidity,cloudiness,windspeed
0,Punta Arenas,3874787,-53.15,-70.92,1579926794,9.00,48.200,87,20,5.70
1,Massaguet,2428231,12.48,15.44,1579926800,14.00,57.200,19,0,4.60
2,Kavieng,2094342,-2.57,150.80,1579926800,28.95,84.110,73,42,7.11
3,Lebu,3883457,-37.62,-73.65,1579926800,12.87,55.166,70,0,3.16
4,Bathsheba,3374083,13.22,-59.52,1579926800,26.00,78.800,78,20,4.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Create a heat map that displays the humidity for every city from the part I of the homework.
#01-Lesson-Plans/06-Python-APIs/3/Activities/07-Stu_Airport_Map/Solved/airport_heatmap.ipynb
# Store latitude and longitude in locations
locations = city_weather_data[["lat", "lon"]]

# Fill NaN values and convert to float
humidity = city_weather_data["humidity"].astype(float)


In [15]:
locations

,lat,lon
0,-53.15,-70.92
1,12.48,15.44
2,-2.57,150.80
3,-37.62,-73.65
4,13.22,-59.52
...,...,...
586,13.07,-59.53
587,36.06,103.79
588,15.00,-83.23
589,-43.53,172.63


In [21]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2,center=(0,0))
#fig = gmaps.figure(zoom_level=2,center=(24.55,45.55))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map